In [210]:
# mnistをCNNで

In [211]:
# ドライブのマウント
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [212]:
%cd /content/drive/MyDrive/kaggle/mnist

/content/drive/MyDrive/kaggle/mnist


In [213]:
import pandas as pd
import numpy as np

In [214]:
# データの読み込み
train = pd.read_csv("data/train.csv")
test = pd.read_csv("data/test.csv")

In [215]:
train.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,pixel11,pixel12,pixel13,pixel14,pixel15,pixel16,pixel17,pixel18,pixel19,pixel20,pixel21,pixel22,pixel23,pixel24,pixel25,pixel26,pixel27,pixel28,pixel29,pixel30,pixel31,pixel32,pixel33,pixel34,pixel35,pixel36,pixel37,pixel38,...,pixel744,pixel745,pixel746,pixel747,pixel748,pixel749,pixel750,pixel751,pixel752,pixel753,pixel754,pixel755,pixel756,pixel757,pixel758,pixel759,pixel760,pixel761,pixel762,pixel763,pixel764,pixel765,pixel766,pixel767,pixel768,pixel769,pixel770,pixel771,pixel772,pixel773,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [216]:
test.head()

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,pixel11,pixel12,pixel13,pixel14,pixel15,pixel16,pixel17,pixel18,pixel19,pixel20,pixel21,pixel22,pixel23,pixel24,pixel25,pixel26,pixel27,pixel28,pixel29,pixel30,pixel31,pixel32,pixel33,pixel34,pixel35,pixel36,pixel37,pixel38,pixel39,...,pixel744,pixel745,pixel746,pixel747,pixel748,pixel749,pixel750,pixel751,pixel752,pixel753,pixel754,pixel755,pixel756,pixel757,pixel758,pixel759,pixel760,pixel761,pixel762,pixel763,pixel764,pixel765,pixel766,pixel767,pixel768,pixel769,pixel770,pixel771,pixel772,pixel773,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [217]:
# データの型を調べる
train.dtypes

label       int64
pixel0      int64
pixel1      int64
pixel2      int64
pixel3      int64
            ...  
pixel779    int64
pixel780    int64
pixel781    int64
pixel782    int64
pixel783    int64
Length: 785, dtype: object

In [218]:
# nullかどうかを調べてから、その数を調べる
train.isnull().sum()

label       0
pixel0      0
pixel1      0
pixel2      0
pixel3      0
           ..
pixel779    0
pixel780    0
pixel781    0
pixel782    0
pixel783    0
Length: 785, dtype: int64

In [219]:
test.isnull().sum()

pixel0      0
pixel1      0
pixel2      0
pixel3      0
pixel4      0
           ..
pixel779    0
pixel780    0
pixel781    0
pixel782    0
pixel783    0
Length: 784, dtype: int64

In [220]:
# データの形状を調整
# 入力を784、出力を10にする
x = train.iloc[:,1:785]
x.head()


,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,pixel11,pixel12,pixel13,pixel14,pixel15,pixel16,pixel17,pixel18,pixel19,pixel20,pixel21,pixel22,pixel23,pixel24,pixel25,pixel26,pixel27,pixel28,pixel29,pixel30,pixel31,pixel32,pixel33,pixel34,pixel35,pixel36,pixel37,pixel38,pixel39,...,pixel744,pixel745,pixel746,pixel747,pixel748,pixel749,pixel750,pixel751,pixel752,pixel753,pixel754,pixel755,pixel756,pixel757,pixel758,pixel759,pixel760,pixel761,pixel762,pixel763,pixel764,pixel765,pixel766,pixel767,pixel768,pixel769,pixel770,pixel771,pixel772,pixel773,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [221]:
# 正規化
x = x / 255

In [222]:
x.head()

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,pixel11,pixel12,pixel13,pixel14,pixel15,pixel16,pixel17,pixel18,pixel19,pixel20,pixel21,pixel22,pixel23,pixel24,pixel25,pixel26,pixel27,pixel28,pixel29,pixel30,pixel31,pixel32,pixel33,pixel34,pixel35,pixel36,pixel37,pixel38,pixel39,...,pixel744,pixel745,pixel746,pixel747,pixel748,pixel749,pixel750,pixel751,pixel752,pixel753,pixel754,pixel755,pixel756,pixel757,pixel758,pixel759,pixel760,pixel761,pixel762,pixel763,pixel764,pixel765,pixel766,pixel767,pixel768,pixel769,pixel770,pixel771,pixel772,pixel773,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [223]:
# Reshape image in 3 dimensions (height = 28px, width = 28px , canal = 1)
x = x.values.reshape(-1,28,28,1)
x

array([[[[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        ...,

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]]],


       [[[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        ...,

        [[0.],
 

In [224]:
# ラベルをone hot エンコーディング
y = pd.get_dummies(train, columns=['label'])
y = y.iloc[:,784:794]
y.head()

,label_0,label_1,label_2,label_3,label_4,label_5,label_6,label_7,label_8,label_9
0,0,1,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0
2,0,1,0,0,0,0,0,0,0,0
3,0,0,0,0,1,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0


In [225]:
"""
# 入力データを28*28にする
# 空のデータフレームを作って28列ごとに行を追加
# 訓練データの１行目を取得
#x_data = x[0:1]
#print(x_data)
# 28個のカラムを取得
# ２行目以降は列の行の名前を変更
data = {}
x_data = {}
x_list = {}
i = 0
s = 1
t = 0

for t in range(42000):
  data[t] = pd.DataFrame()
  x_data[t] = x[t: t+1]
  for s in range(28):
    x_list[s] = x_data[t].iloc[:,i:i+28]
    i += 28
    print(x_list[s])
    for num2 in range(28):
      x_list[s] = x_list[s].rename(columns={f'pixel{num2+(28*s)}': f'pixel{num2}'}, index={t: s})
    print(x_list[s])
    data[t] = data[t].append(x_list[s])
    s += 1
  print(data[t])
  i = 0
  s = 0
"""

"\n# 入力データを28*28にする\n# 空のデータフレームを作って28列ごとに行を追加\n# 訓練データの１行目を取得\n#x_data = x[0:1]\n#print(x_data)\n# 28個のカラムを取得\n# ２行目以降は列の行の名前を変更\ndata = {}\nx_data = {}\nx_list = {}\ni = 0\ns = 1\nt = 0\n\nfor t in range(42000):\n  data[t] = pd.DataFrame()\n  x_data[t] = x[t: t+1]\n  for s in range(28):\n    x_list[s] = x_data[t].iloc[:,i:i+28]\n    i += 28\n    print(x_list[s])\n    for num2 in range(28):\n      x_list[s] = x_list[s].rename(columns={f'pixel{num2+(28*s)}': f'pixel{num2}'}, index={t: s})\n    print(x_list[s])\n    data[t] = data[t].append(x_list[s])\n    s += 1\n  print(data[t])\n  i = 0\n  s = 0\n"

In [226]:
# 訓練データとテストデータに分割
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=0)

In [227]:
# CNNのモデルを作成
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D

In [228]:
# モデルを定義
model = Sequential()
#model.add(Conv2D(filters = 30, kernel_size = (5,5),padding = 'Same', activation ='relu', input_shape = (28,28,1)))
model.add(Conv2D(30, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.1))
model.add(Conv2D(30, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.1))
model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dense(10, activation='softmax'))

In [229]:
sgd = keras.optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

model.fit(x_train, y_train, batch_size=100, epochs=50)
#score = model.evaluate(x_test, y_test, batch_size=32)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


Epoch 1/50
315/315 [==============================] - 24s 74ms/step - loss: 0.5404 - accuracy: 0.8303
Epoch 2/50
315/315 [==============================] - 23s 73ms/step - loss: 0.1486 - accuracy: 0.9534
Epoch 3/50
315/315 [==============================] - 25s 80ms/step - loss: 0.1101 - accuracy: 0.9647
Epoch 4/50
315/315 [==============================] - 23s 72ms/step - loss: 0.0902 - accuracy: 0.9703
Epoch 5/50
315/315 [==============================] - 23s 72ms/step - loss: 0.0785 - accuracy: 0.9745
Epoch 6/50
315/315 [==============================] - 23s 72ms/step - loss: 0.0691 - accuracy: 0.9779
Epoch 7/50
315/315 [==============================] - 23s 72ms/step - loss: 0.0617 - accuracy: 0.9803
Epoch 8/50
315/315 [==============================] - 23s 72ms/step - loss: 0.0552 - accuracy: 0.9819
Epoch 9/50
315/315 [==============================] - 22s 71ms/step - loss: 0.0516 - accuracy: 0.9834
Epoch 10/50
315/315 [==============================] - 23s 72ms/step - loss: 0.045

In [230]:
#model.fit(x_test, y_test, batch_size=100, epochs=50)

In [245]:
model.evaluate(x_test, y_test)

329/329 [==============================] - 3s 10ms/step - loss: 0.0434 - accuracy: 0.9898


[0.04336138442158699, 0.989809513092041]

In [246]:
#submit
#x_submit = test.iloc[:,1:785]
x_submit = test
x_submit = x_submit/255
x_submit = x_submit.values.reshape(-1,28,28,1)
#予測したデータを入れるためのデータフレーム
y_submit = pd.DataFrame()
for i in range(10):
  y_submit[f'label_{i}'] = []
#y_submit = y_submit[['label_0', 'label_1', 'label_2', 'label_3', 'label_4', 'label_5', 'label_6', 'label_7', 'label_8', 'label_9']]
#data[t] = data[t].append(x_)
#y_submit = x_submit.idxmax(axis=1)
y_submit['ImageId'] = test.index + 1
y_submit.head()

,label_0,label_1,label_2,label_3,label_4,label_5,label_6,label_7,label_8,label_9,ImageId
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5


In [247]:
y_submit[['label_0', 'label_1', 'label_2', 'label_3', 'label_4', 'label_5', 'label_6', 'label_7', 'label_8', 'label_9']] = model.predict(x_submit)
y_submit.head()

,label_0,label_1,label_2,label_3,label_4,label_5,label_6,label_7,label_8,label_9,ImageId
0,7.705347e-14,9.857174e-13,1.000000e+00,1.525680e-10,2.791094e-12,6.162846e-15,3.226082e-15,2.461753e-10,6.891140e-12,1.445035e-15,1
1,1.000000e+00,2.824324e-11,3.961481e-10,3.377205e-15,4.040836e-12,3.968237e-09,1.249118e-09,2.390835e-08,3.621596e-15,2.060127e-10,2
2,3.838362e-16,7.725261e-12,9.837532e-11,5.413068e-10,2.883942e-07,6.589717e-12,5.725498e-15,6.132114e-10,3.436718e-07,9.999994e-01,3
3,9.663288e-03,1.954936e-14,6.617260e-07,1.310347e-07,1.280836e-07,1.578286e-11,9.106354e-09,5.030364e-07,2.256065e-07,9.903350e-01,4
4,2.192927e-11,9.426232e-12,6.765672e-05,9.999322e-01,2.395421e-15,3.737640e-10,2.523669e-14,2.924232e-10,1.407166e-07,6.533259e-12,5


In [248]:
#y_submit_1 = y_submit.drop(labels = ["ImageId"])
#y_submit_1 = y_submit.iloc[:,0:0]
#imageidを除外したい
#y_submit = y_submit_1.idxmax(axis=1)
y_submit_a = y_submit.drop('ImageId', axis=1)
y_submit_a.head()

,label_0,label_1,label_2,label_3,label_4,label_5,label_6,label_7,label_8,label_9
0,7.705347e-14,9.857174e-13,1.000000e+00,1.525680e-10,2.791094e-12,6.162846e-15,3.226082e-15,2.461753e-10,6.891140e-12,1.445035e-15
1,1.000000e+00,2.824324e-11,3.961481e-10,3.377205e-15,4.040836e-12,3.968237e-09,1.249118e-09,2.390835e-08,3.621596e-15,2.060127e-10
2,3.838362e-16,7.725261e-12,9.837532e-11,5.413068e-10,2.883942e-07,6.589717e-12,5.725498e-15,6.132114e-10,3.436718e-07,9.999994e-01
3,9.663288e-03,1.954936e-14,6.617260e-07,1.310347e-07,1.280836e-07,1.578286e-11,9.106354e-09,5.030364e-07,2.256065e-07,9.903350e-01
4,2.192927e-11,9.426232e-12,6.765672e-05,9.999322e-01,2.395421e-15,3.737640e-10,2.523669e-14,2.924232e-10,1.407166e-07,6.533259e-12


In [255]:
y_submit['label'] = y_submit_a.idxmax(axis=1)
y_submit['label']

0        label_2
1        label_0
2        label_9
3        label_9
4        label_3
          ...   
27995    label_9
27996    label_7
27997    label_3
27998    label_9
27999    label_2
Name: label, Length: 28000, dtype: object

In [256]:
#y_submit.max(axis='columns')
#y_submit['label'] = y_submit_a.max(axis=1)
#y_submit_a.idxmax(axis=1)
#y_submit_a.iloc[[num]].idxmax(axis=1)
#label = y_submit_a.iloc[[num]].idxmax(axis=1)
#label = label.values[0]

for num in range(len(y_submit['ImageId'])):
  L = y_submit_a.iloc[[num]].idxmax(axis=1)
  L = L.values[0]
  if  L == 'label_0':
    y_submit['label'][num] = 0
    print('label_0')
  elif L == 'label_1':
    y_submit['label'][num] = 1
    print('label_1')
  elif L == 'label_2':
    y_submit['label'][num] = 2
    print('label_2')
  elif L == 'label_3':
    y_submit['label'][num] = 3
    print('label_3')
  elif L == 'label_4':
    y_submit['label'][num] = 4
    print('label_4')
  elif L == 'label_5':
    y_submit['label'][num] = 5
    print('label_5')
  elif L == 'label_6':
    y_submit['label'][num] = 6
    print('label_6')
  elif L == 'label_7':
    y_submit['label'][num] = 7
    print('label_7')
  elif L == 'label_8':
    y_submit['label'][num] = 8
    print('label_8')
  elif L == 'label_9':
    y_submit['label'][num] = 9
    print('label_9')
  print(num)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: SettingWithCopyWarning:

ストリーミング出力は最後の 5000 行に切り捨てられました。
label_0
25500
label_7
25501
label_1
25502
label_7
25503
label_1
25504
label_2
25505
label_7
25506
label_4
25507
label_0
25508
label_9
25509
label_6
25510
label_2
25511
label_6
25512
label_3
25513
label_2
25514
label_6
25515
label_7
25516
label_5
25517
label_1
25518
label_0
25519
label_4
25520
label_3
25521
label_5
25522
label_9
25523
label_1
25524
label_4
25525
label_3
25526
label_1
25527
label_3
25528
label_5
25529
label_5
25530
label_7
25531
label_6
25532
label_9
25533
label_6
25534
label_7
25535
label_5
25536
label_6
25537
label_9
25538
label_8
25539
label_5
25540
label_1
25541
label_3
25542
label_0
25543
label_6
25544
label_7
25545
label_8
25546
label_7
25547
label_6
25548
label_7
25549
label_4
25550
label_7
25551
label_6
25552
label_3
25553
label_7
25554
label_3
25555
label_6
25556
label_1
25557
label_8
25558
label_2
25559
label_7
25560
label_8
25561
label_7
25562
label_9
25563
label_4
25564
label_1
25565
label_8
25566
label_8
25567
label_7
25568
la

In [257]:
y_submit.head()

,label_0,label_1,label_2,label_3,label_4,label_5,label_6,label_7,label_8,label_9,ImageId,label
0,7.705347e-14,9.857174e-13,1.000000e+00,1.525680e-10,2.791094e-12,6.162846e-15,3.226082e-15,2.461753e-10,6.891140e-12,1.445035e-15,1,2
1,1.000000e+00,2.824324e-11,3.961481e-10,3.377205e-15,4.040836e-12,3.968237e-09,1.249118e-09,2.390835e-08,3.621596e-15,2.060127e-10,2,0
2,3.838362e-16,7.725261e-12,9.837532e-11,5.413068e-10,2.883942e-07,6.589717e-12,5.725498e-15,6.132114e-10,3.436718e-07,9.999994e-01,3,9
3,9.663288e-03,1.954936e-14,6.617260e-07,1.310347e-07,1.280836e-07,1.578286e-11,9.106354e-09,5.030364e-07,2.256065e-07,9.903350e-01,4,9
4,2.192927e-11,9.426232e-12,6.765672e-05,9.999322e-01,2.395421e-15,3.737640e-10,2.523669e-14,2.924232e-10,1.407166e-07,6.533259e-12,5,3


In [258]:
y_submit_b = y_submit.iloc[:,10:12]
y_submit_b.head()

,ImageId,label
0,1,2
1,2,0
2,3,9
3,4,9
4,5,3


In [259]:
# csvファイルを作成
y_submit_b.to_csv('submission/submit08.csv', index=False)